#Import Libraries

In [111]:
import gdown
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly.express as px
from folium import plugins
from folium.plugins import HeatMap
from scipy.stats import pointbiserialr

#Import & Load Dataset

In [85]:
# Import Dataset
dataset_link = 'https://drive.google.com/drive/folders/1wLAk6IsUNCWhGENAmAUnmXZxLiiv7m4q?usp=sharing'
gdown.download_folder(dataset_link, quiet=True, use_cookies=False)

# Path Dataset
path = '/content/Dataset/'

# Get the data from Dataset Folder
csv_files = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.csv')]
df = {}

# Load each CSV file into a DataFrame
for file_name in csv_files:
    base_name = os.path.basename(file_name).replace('.csv', '')
    print(f"Load Path: {file_name} into DataFrame: {base_name}..")

    # Load the file and store it
    df[base_name] = pd.read_csv(file_name)

print("\nList Dataset")
print(df.keys())

Load Path: /content/Dataset/reviews.csv into DataFrame: reviews..
Load Path: /content/Dataset/listings.csv into DataFrame: listings..
Load Path: /content/Dataset/calendar.csv into DataFrame: calendar..

List Dataset
dict_keys(['reviews', 'listings', 'calendar'])


##Dataset Reviews

In [86]:
df['reviews'].head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


##Dataset Calendar

In [88]:
df['calendar'].head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


##Dataset Listings

In [90]:
df['listings'].head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


#Descriptive Statistics

* Apakah ada kolom dengan tipe data kurang sesuai, atau nama kolom dan isinya
kurang sesuai?
* Apakah ada kolom yang memiliki nilai kosong? Jika ada, apa saja?
* Apakah ada kolom yang memiliki nilai summary agak aneh?
(min/mean/median/max/unique/top/freq)


In [92]:
reviews = pd.read_csv('/content/Dataset/reviews.csv')
calendar = pd.read_csv('/content/Dataset/calendar.csv')
listings = pd.read_csv('/content/Dataset/listings.csv')

##Dataset Reviews

In [99]:
# Memeriksa informasi umum pada reviews
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   listing_id     84849 non-null  int64 
 1   id             84849 non-null  int64 
 2   date           84849 non-null  object
 3   reviewer_id    84849 non-null  int64 
 4   reviewer_name  84849 non-null  object
 5   comments       84831 non-null  object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


Kolom-kolom id sebaiknya diubah menjadi string (kategorikal) untuk mempermudah pengolahan dan menghindari masalah potensial terkait format yang tidak sesuai.


In [102]:
# Mengubah data type id menjadi object
reviews['listing_id'] = reviews['listing_id'].astype(str)
reviews['id'] = reviews['id'].astype(str)

In [93]:
# Memeriksa data kosong pada reviews
reviews.isnull().sum()

,0
listing_id,0
id,0
date,0
reviewer_id,0
reviewer_name,0
comments,18


Terdapat 18 baris kosong pada kolom comments yang harus diolah.

In [94]:
# Memeriksa data duplikat pada reviews
reviews.duplicated().sum()

0

Dataset reviews tidak memiliki nilai duplikat.

##Dataset Calendar

In [100]:
# Memeriksa informasi umum pada calendar
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1393570 non-null  int64 
 1   date        1393570 non-null  object
 2   available   1393570 non-null  object
 3   price       934542 non-null   object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


Kolom id sebaiknya diubah menjadi string (kategorikal) untuk mempermudah pengolahan dan menghindari masalah potensial terkait format yang tidak sesuai.

In [104]:
# Mengubah data type id menjadi object
calendar['listing_id'] = calendar['listing_id'].astype(str)

In [95]:
# Memeriksa data kosong pada calendar
calendar.isnull().sum()

,0
listing_id,0
date,0
available,0
price,459028


Terdapat banyak sekali baris kosong pada kolom price, sebaiknya menghapus kolom price karena tidak efisien.

In [96]:
# Memeriksa data duplikat pada calendar
calendar.duplicated().sum()

0

Dataset calendar tidak memiliki nilai duplikat.

##Dataset Listings

In [101]:
# Memeriksa informasi umum pada listings
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_url                       3818 non-null   object 
 2   scrape_id                         3818 non-null   int64  
 3   last_scraped                      3818 non-null   object 
 4   name                              3818 non-null   object 
 5   summary                           3641 non-null   object 
 6   space                             3249 non-null   object 
 7   description                       3818 non-null   object 
 8   experiences_offered               3818 non-null   object 
 9   neighborhood_overview             2786 non-null   object 
 10  notes                             2212 non-null   object 
 11  transit                           2884 non-null   object 
 12  thumbn

* Mengubah nama id menjadi listing_id agar bisa menggabukan dataset dengan reviews dan calendar
* Membuat listing baru yang hanya berisi kolom penting untuk dianalisis
* Mengubah kolom-kolom id menjadi string untuk mempermudah pengolahan
* Mengubah beberapa data type kolom object menjadi numerik



In [105]:
# Mengubah nama kolom id
listings = listings.rename(columns={'id':'listing_id'})

In [106]:
# Mengubah data type id menjadi object
listings['listing_id'] = listings['listing_id'].astype(str)
listings['scrape_id'] = listings['scrape_id'].astype(str)

In [97]:
# Memeriksa data kosong pada listings
listings.isnull().sum()[listings.isnull().sum() != 0]

,0
summary,177
space,569
neighborhood_overview,1032
notes,1606
transit,934
thumbnail_url,320
medium_url,320
xl_picture_url,320
host_name,2
host_since,2


* Mengisi kolom kosong numerik dengan median atau mean tergantung analisis outlier
* Mengisi kolom kosong kategorikal dengan data terbanyak atau unknown
* Menghapus license dan square_feet karena terlalu banyak baris kosong

In [98]:
# Memeriksa data duplikat pada listings
listings.duplicated().sum()

0

In [107]:
listings.describe()

,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3816.000000,3816.000000,3818.000000,3818.000000,3818.000000,3802.000000,3812.000000,3817.000000,97.000000,...,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3162.000000,0.0,3818.000000,3191.000000
mean,1.578556e+07,7.157757,7.157757,47.628961,-122.333103,3.349398,1.259469,1.307712,1.735394,854.618557,...,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,NaN,2.946307,2.078919
std,1.458382e+07,28.628149,28.628149,0.043052,0.031745,1.977599,0.590369,0.883395,1.139480,671.404893,...,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,0.750259,NaN,5.893029,1.822348
min,4.193000e+03,1.000000,1.000000,47.505088,-122.417219,1.000000,0.000000,0.000000,1.000000,0.000000,...,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,NaN,1.000000,0.020000
25%,3.275204e+06,1.000000,1.000000,47.609418,-122.354320,2.000000,1.000000,1.000000,1.000000,420.000000,...,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.695000
50%,1.055814e+07,1.000000,1.000000,47.623601,-122.328874,3.000000,1.000000,1.000000,1.000000,750.000000,...,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,1.000000,1.540000
75%,2.590309e+07,3.000000,3.000000,47.662694,-122.310800,4.000000,1.000000,2.000000,2.000000,1200.000000,...,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,2.000000,3.000000
max,5.320861e+07,502.000000,502.000000,47.733358,-122.240607,16.000000,8.000000,7.000000,15.000000,3000.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,37.000000,12.150000


Dataset listings tidak memiliki nilai duplikat

#Univariate Analysis

Gunakan visualisasi untuk melihat distribusi masing-masing kolom (feature maupun
target). Tuliskan hasil observasinya, misalnya jika ada suatu kolom yang distribusinya
menarik (misal skewed, bimodal, ada outlier, ada nilai yang mendominasi, kategorinya
terlalu banyak, dsb). Jelaskan juga apa yang harus di-follow up saat data pre-processing.

#Multivariate Analysis

Lakukan multivariate analysis (seperti correlation heatmap dan category plots, sesuai yang
diajarkan di kelas). Tuliskan hasil observasinya, seperti:
* Bagaimana korelasi antara masing-masing feature dan label. Kira-kira feature mana
saja yang paling relevan dan harus dipertahankan?
* Bagaimana korelasi antar-feature, apakah ada pola yang menarik? Apa yang perlu
dilakukan terhadap feature itu?

##Korelasi Antara Feature & Target

##Korelasi Antara Feature

#Data Cleansing (Lakukan analisis terlebih dahulu)

##Dataset Reviews

In [ ]:
# Menghapus kolom kosong pada reviews
reviews = reviews.dropna(axis = 0, subset = ['comments'], how = 'any')

In [ ]:
# Memeriksa data kosong pada reviews setelah dihapus
reviews.isnull().sum()

* Dataset reviews hanya memiliki data kosong dan tidak memiliki data yang duplikat, jadi cukup hanya dengan menghapus data yang kosong.
* Tidak banyak yang perlu diubah dari kolom reviews karena sudah sesuai.

##Dataset Calendar

In [ ]:
# Menghapus kolom kosong pada calendar
calendar = calendar.dropna(axis = 0, subset = ['price'], how = 'any')

In [ ]:
# Memeriksa data kosong pada calendar setelah dihapus
calendar.isnull().sum()

In [ ]:
# Mengubah data type calendar menjadi datetime
calendar['date'] = pd.to_datetime(calendar['date'])

# Memisahkan kolom date menjadi day, month, dan year
calendar['day'] = calendar.date.dt.day
calendar['month'] = calendar.date.dt.month
calendar['year'] = calendar.date.dt.year

# Menghapus lambang dolar pada price
calendar['price'] = pd.to_numeric(calendar['price'].apply(lambda x: str(x).replace('$', '').replace(',', '')), errors='coerce')

# Mengubah available menjadi tipe data boolean
calendar['available'] = calendar['available'].map({'t': True, 'f': False})

In [ ]:
# Melihat dataset calendar setelah diproses
calendar.head()

* Dataset calendar memiliki data kosong pada kolom price dan banyak sekali baris yang kosong, sehingga perlu dihapus untuk kelancaran analisis.
* Kolom price memiliki simbol "$" dan koma (",") sehingga tidak cocok sebagai object. Lalu konversi kolom price ke tipe numerik setelah membersihkan simbol tersebut.
* Kolom date sebaiknya memiliki tipe data datetime daripada object. Setelah diubah pisahkan day, month, dan year untuk mendapatkan insight yang lebih luas.
* Kolom available masih berisi nilai t/f (true/false) dalam bentuk string, dikonversi ke tipe boolean.

##Dataset Listings

In [ ]:
# Mengganti dan membersihkan datatype yang kurang sesuai
listings['price'] = pd.to_numeric(listings['price'].apply(lambda x: str(x).replace('$', '').replace(',', '')), errors='coerce')
listings['monthly_price'] = pd.to_numeric(listings['monthly_price'].apply(lambda x: str(x).replace('$', '').replace(',', '')), errors='coerce')
listings['weekly_price'] = pd.to_numeric(listings['weekly_price'].apply(lambda x: str(x).replace('$', '').replace(',', '')), errors='coerce')
listings['cleaning_fee'] = pd.to_numeric(listings['cleaning_fee'].apply(lambda x: str(x).replace('$', '').replace(',', '')), errors='coerce')
listings['security_deposit'] = pd.to_numeric(listings['security_deposit'].apply(lambda x: str(x).replace('$', '').replace(',', '')), errors='coerce')
listings['extra_people'] = pd.to_numeric(listings['extra_people'].apply(lambda x: str(x).replace('$', '').replace(',', '')), errors='coerce')

Menghapus lambang dolar pada kolom yang memiliki lambang dolar dan mengubahnya menjadi numerik untuk memudahkan analisis dan visualisasi

In [ ]:
# Mengisi nilai kosong untuk kolom kategori
listings['summary'] = listings['summary'].fillna("Tidak tersedia")
listings['space'] = listings['space'].fillna("Tidak tersedia")
listings['neighborhood_overview'] = listings['neighborhood_overview'].fillna("Tidak tersedia")
listings['notes'] = listings['notes'].fillna("Tidak tersedia")
listings['transit'] = listings['transit'].fillna("Tidak tersedia")

Mengisi kolom kategorik yang kosong menjadi "Tidak tersedia"

In [ ]:
# Mengisi nilai kosong untuk kolom numerik
listings['bathrooms'] = listings['bathrooms'].fillna(listings['bathrooms'].median())
listings['bedrooms'] = listings['bedrooms'].fillna(listings['bedrooms'].median())
listings['review_scores_rating'] = listings['review_scores_rating'].fillna(listings['review_scores_rating'].median())
listings['reviews_per_month'] = listings['reviews_per_month'].fillna(listings['reviews_per_month'].mean())
listings['square_feet'] = listings['square_feet'].fillna(0)
listings['security_deposit'] = listings['security_deposit'].fillna(0)
listings['cleaning_fee'] = listings['cleaning_fee'].fillna(0)

Mengisi kolom numerik
* median digunakan untuk kolom yang mempunyai nilai outlier
* mean digunakan untuk kolom yang memiliki distribusi tidak terlalu miring
* 0 digunakan untuk kolom yang masuk akal diisi dengan data kosong/nol

In [ ]:
# Mengisi kolom URL dengan default
listings[['thumbnail_url', 'medium_url', 'xl_picture_url']] = listings[['thumbnail_url', 'medium_url', 'xl_picture_url']].fillna("No Image")

Jika nilai kosong, isi dengan string "No Image"


In [ ]:
# Mengubah typedata rate menjadi numerik
listings['host_response_rate'] = listings['host_response_rate'].str.rstrip('%').astype(float)
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.rstrip('%').astype(float)

Karena ada simbol % sebaiknya diubah menjadi numerik, agar mendapat pengolahan lebih lanjut

In [ ]:
# Menghapus kolom yang terlalu banyak memiliki nilai kosong
listings = listings.drop(columns=['license'])

Masih banyak kolom yang memiliki nilai kosong jadi sebaiknya hanya mengambil beberapa kolom saja dari listings yang sesuai dengan goal dan objective yang ingin dicapai.